In [1]:
import pandas
import datetime
import numpy
from scipy.optimize import curve_fit
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from matplotlib import ticker
%matplotlib inline
from matplotlib import rcParams

In [2]:
datFileName = "../data/processedSaps.txt"
sapsDataDF = pandas.read_csv(datFileName, sep=' ')
# # add dst_bins
dstBins = [ -150, -75, -50, -25, -10, 10 ]
sapsDataDF = pandas.concat( [ sapsDataDF, \
                    pandas.cut( sapsDataDF["dst_index"], \
                               bins=dstBins ) ], axis=1 )
sapsDataDF.columns = [ "dateStr", "sapsLat", "sapsMLT", \
                      "sapsVel", "radId", "poesLat", "poesMLT", \
                      "dst_date", "dst_index", "time", "dst_bin" ]
sapsDataDF = sapsDataDF.drop(["poesLat", "poesMLT"], 1)
sapsDataDF["sapsMLTRounded"] = sapsDataDF["sapsMLT"].map(lambda x: round(x) )
sapsDataDF.head()

,dateStr,sapsLat,sapsMLT,sapsVel,radId,dst_date,dst_index,time,dst_bin,sapsMLTRounded
0,20110107,56.5,17.7543,308.2077,33.0,2011-01-07 00:00:00,-18.0,0,"(-25, -10]",18.0
1,20110107,55.5,18.0147,224.1588,33.0,2011-01-07 00:00:00,-18.0,0,"(-25, -10]",18.0
2,20110107,56.5,17.8749,307.4328,33.0,2011-01-07 00:00:00,-18.0,0,"(-25, -10]",18.0
3,20110107,55.5,18.1324,222.4787,33.0,2011-01-07 00:00:00,-18.0,0,"(-25, -10]",18.0
4,20110107,56.5,17.9955,305.4201,33.0,2011-01-07 00:00:00,-18.0,0,"(-25, -10]",18.0


In [3]:
sapsNumRadsSer = sapsDataDF.groupby(["dateStr"]).agg( {"radId": pandas.Series.nunique} )

In [4]:
sapsTestDate = sapsDataDF[ sapsDataDF["dateStr"] == 20120618 ].reset_index(drop=True)
sapsTestDate.head()

,dateStr,sapsLat,sapsMLT,sapsVel,radId,dst_date,dst_index,time,dst_bin,sapsMLTRounded
0,20120618,57.5,20.4282,223.5077,32.0,2012-06-18 00:00:00,-52.0,30,"(-75, -50]",20.0
1,20120618,58.5,19.6274,206.5899,207.0,2012-06-18 01:00:00,-53.0,100,"(-75, -50]",20.0
2,20120618,58.5,19.7550,204.4557,207.0,2012-06-18 01:00:00,-53.0,100,"(-75, -50]",20.0
3,20120618,57.5,19.5212,309.6188,207.0,2012-06-18 01:00:00,-53.0,100,"(-75, -50]",20.0
4,20120618,56.5,19.6662,391.7483,207.0,2012-06-18 01:00:00,-53.0,100,"(-75, -50]",20.0
